In [1]:
from final_target import get_target_list_db, get_target
from final_predict import stock_predict
from final_model import LstmNetwork

from keras.models import load_model
import tensorflow as tf
import os

# tf gpu 메모리 관련 코드 
gpus = tf.config.list_physical_devices(device_type = 'GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

kospi_list, kosdaq_list = get_target_list_db()
investing_df = get_target(path=r'C:\big15\project-dev\trading\download\target')




# 소형주 6개월 학습
ls_small_six_month = LstmNetwork('소형주', kosdaq_list, '6개월', 140, 0.03)


실시간 트레이딩 기준 날짜 : 20221220
크롤링 실행 중
크롤링 실행
dddddddddddd
True


In [2]:
model_small_six_month = ls_small_six_month.model_
col_list_small_six_month = ls_small_six_month.col_list

# stock_predict(kosdaq_list, investing_df, col_list_small_six_month, model_small_six_month)

In [3]:
from final_dbconnect import DBConnection_trading, DBConnection_predict

from sklearn.preprocessing import MaxAbsScaler
from pandas.tseries.offsets import BDay
from datetime import date, datetime

import pandas as pd
import time


In [4]:
today = str(date.today()).replace('-','')
yesterday=str(date.today() - BDay(1)).replace('-','').split(' ')[0]

def get_pymysql_traidng_table_check(table_schema, code, conn):
    # 현재 DB 내 존재하는 테이블 존재 여부 확인
    sql = f"SELECT 1 FROM Information_schema.tables  WHERE table_schema = '{table_schema}' AND table_name = '{code}_{today}'"

    cur = conn.cursor()
    count = cur.execute(sql)

    return count

def get_pymysql_day_stock(conn, code, yesterday, investing_df):

    code = code[-6:]
    sql = f"SELECT 전일대비, 상장주식수, 시가총액, 외국인현보유수량, 외국인현보유비율, 기관순매수량, 기관누적순매수량, 년, 월, 일 FROM stock_info.`{code}` WHERE 날짜={yesterday} ORDER BY 시간 DESC LIMIT 1"
    
    result = conn.execute(sql)

    target_df = pd.DataFrame(result.fetchall())
    # target_df.set_index('날짜', inplace=True)
    # target_df.rename(index={int(yesterday):today}, inplace=True)
    target_df = pd.concat([target_df, investing_df], axis=1)

    return target_df

In [5]:
def predict_stock(pred_df, col_list):
    min_abs_scaler = MaxAbsScaler()
    c_list = list(col_list.index)
    each_target_df = table_df[c_list]
    X_pred_sc = min_abs_scaler.fit_transform(each_target_df)
    X_pred = X_pred_sc.reshape(X_pred_sc.shape[0], model.input.shape[1], 1)

    predict = model.predict(X_pred)
    
    return predict

In [ ]:
stock_list = kosdaq_list
col_list = col_list_small_six_month
model = model_small_six_month
pred_df = pd.DataFrame()
time_cnt = 0

while True:
        now = datetime.now()
        t = time.time()
        if (now.minute == 30) & (now.hour == 15):
            break
        elif (now.hour < 9) | (now.hour > 16):
            time_cnt += 1
            if time_cnt == 10:
                print('박대기중~~~~~~~')
                time_cnt = 0
            time.sleep(1)
        else:
                   
            for code in stock_list:
               
                cnt = 0
                count = 0
                while True:
                    cnt += 1
                    if cnt == 10:
                        print('진행중~~~~~~~')
                        cnt = 0
                    count = get_pymysql_traidng_table_check('trading_data',code, DBConnection_trading().get_pymysql_connection())
                    if count == 1:
                        break
                    time.sleep(1)
                # 전날 일봉 데이터와 investing data
                
                day_stock_investing_df = get_pymysql_day_stock(DBConnection_trading().get_sqlalchemy_connect_ip(), code, yesterday, investing_df)
                
                # day_stock_investing_df.reset_index(drop=True, inplace=True)
                try:
                    sql = f"SELECT 시간, 시가, 고가, 저가, 종가, 거래량, 거래대금, 누적체결매도수량, 누적체결매수수량, 년, 월, 일 FROM trading_data.{code}_{today} ORDER BY 시간 DESC LIMIT 1"
                    table_data = DBConnection_trading().get_sqlalchemy_connect_ip().execute(sql) 
                    table_df = pd.DataFrame(table_data.fetchall())  # DB내 테이블을 DF로 변환
                
                    table_df = pd.concat([table_df, day_stock_investing_df], axis=1 )
                    
                    predict_ = predict_stock(table_df, col_list)
                    
                    predict_df = pd.DataFrame(predict_)
                    predict_df['비교'] = (predict_df[0] - predict_df[1])
                    predict_df['id'] = t
                    pred_df.append(predict_df)
                    
                    pred_cnt =  get_pymysql_traidng_table_check('predict_data',code, DBConnection_trading().get_pymysql_connection())
                    time.sleep(0.2)
                    if pred_cnt == 0:
                        predict_df.to_sql(name='{0}_{1}'.format(code, yesterday), con=DBConnection_predict().get_sqlalchemy_connect_ip(), if_exists='replace', index=False)
                    else:
                        predict_df.to_sql(name='{0}_{1}'.format(code, yesterday), con=DBConnection_predict().get_sqlalchemy_connect_ip(), if_exists='append', index=False)
                except:
                        print('SQL 에러 발생')

In [ ]:
stock_list = ['207760','065950','366030','073570','066410','005860','013720','023440','226330','028300612500']


In [ ]:
col_list_small_six_month = ls_small_six_month.col_list

In [ ]:
print(col_list_small_six_month)

시간                     -0.200079
누적체결매수수량                0.081746
전일대비                    0.032197
기관순매수량                 -0.059900
기관누적순매수량               -0.056364
JPYKRW내역_거래량            0.070113
SETIndex내역_변동          -0.040960
SPBMVIPC내역_거래량          0.042222
VN30내역_거래량              0.044981
WTI유선물내역_거래량           -0.060008
WTI유선물내역_변동            -0.043279
구리선물내역_거래량              0.037497
구리선물내역_변동              -0.058747
금선물내역_거래량               0.090632
나스닥종합지수내역_거래량           0.044260
다우존스내역_거래량              0.046007
대만가권내역_거래량              0.068177
독일1년채권수익율내역_변동         -0.034842
독일9개월채권수익율내역_고가         0.031123
미국소맥선물내역_변동            -0.052664
벨기에BEL내역_거래량            0.072650
브라질보베스파내역_거래량           0.036642
브라질보베스파내역_변동           -0.059360
사우디아라비아Tadawul내역_거래량    0.059909
사우디아라비아Tadawul내역_변동     0.052540
스웨덴OMXS내역_거래량           0.039097
스위스6개월채권수익율내역_변동       -0.083119
스위스SMI내역_거래량            0.050211
영국FTSE내역_거래량            0.057320
이탈리아FTSEMIB내역_거래량       0.033271
일본1개월채권수익율

In [ ]:
col_list

NameError: name 'col_list' is not defined

In [1]:
from final_target import get_target_list_db, get_target
from final_predict import stock_predict
from final_model import LstmNetwork

from keras.models import load_model
import tensorflow as tf
import os

# tf gpu 메모리 관련 코드 
gpus = tf.config.list_physical_devices(device_type = 'GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

kospi_list, kosdaq_list = get_target_list_db()
investing_df = get_target(path=r'C:\big15\project-dev\trading\download\target')

# # 대형주 1년 학습
# ls_big_year = LstmNetwork('대형주', kospi_list, '1년', 280, 0.02)
# col_list_big_year = ls_big_year.complete_corr
# modelpath_big_year = ls_big_year.LstmModel()
# model_big_year = load_model(modelpath_big_year)

# # 대형주 1년 트레이딩
# stock_predict(kospi_list, investing_df, col_list_big_year, model_big_year)



# # 대형주 6개월 학습
# ls_big_six_month = LstmNetwork('대형주', kospi_list, '6개월', 140, 0.04 )
# col_list_big_six_month = ls_big_six_month.complete_corr
# modelpath_big_six_month = ls_big_six_month.LstmModel()
# model_big_six_month = load_model(modelpath_big_six_month)

# # 대형주 6개월 트레이딩
# stock_predict(kospi_list, investing_df, col_list_big_six_month, model_big_six_month)



# # 소형주 1년 학습
# ls_small_year = LstmNetwork('소형주', kosdaq_list, '1년', 280, 0.03)
# col_list_small_year = ls_small_year.complete_corr
# modelpath_small_year = ls_small_year.LstmModel()
# model_small_year = load_model(modelpath_small_year)

# # 소형주 1년 트레이딩
# stock_predict(kosdaq_list, investing_df, col_list_small_year, model_small_year)



# # 소형주 6개월 학습
# ls_small_six_month = LstmNetwork('소형주', kosdaq_list, '6개월', 140, 0.03)
# col_list_small_six_month = ls_small_six_month.col_list
# model_small_six_month = ls_small_six_month.model_weight

# # with open( './pickle/pickle_corr_complete/소형주_20221216_6개월_10개_0.03.pkl', 'rb') as p:
# #     col_list_small_six_month = pickle.load(p)
# # modelpath_small_six_month = './model/소형주_20221216_6개월_lstm_30ep_64bs_5pa_0.03newcor.hdf5'
# # model_small_six_month = load_model(modelpath_small_six_month)

# # 소형주 6개월 트레이딩
# stock_predict(kosdaq_list, investing_df, col_list_small_six_month, model_small_six_month)

실시간 트레이딩 기준 날짜 : 20221221
크롤링 실행 중
크롤링 실행


In [2]:
investing_df

,AEDKRW내역_종가,AEDKRW내역_오픈,AEDKRW내역_고가,AEDKRW내역_저가,AEDKRW내역_거래량,AEDKRW내역_변동,ATX내역_종가,ATX내역_오픈,ATX내역_고가,ATX내역_저가,...,헝가리BUX내역_고가,헝가리BUX내역_저가,헝가리BUX내역_거래량,헝가리BUX내역_변동,호주SPASX내역_종가,호주SPASX내역_오픈,호주SPASX내역_고가,호주SPASX내역_저가,호주SPASX내역_거래량,호주SPASX내역_변동
0,351.36,354.41,355.3,350.04,0.0,-0.82,3066.84,3059.12,3099.53,3057.85,...,45494.53,44800.37,0.0,0.45,7024.3,7133.9,7133.9,7019.5,678280000,-1.54


In [2]:
import pandas as pd

In [3]:
investing_df = pd.read_csv('./download/investing_df/investing_20221221.csv', encoding='utf-8-sig', index_col=0)

In [4]:
investing_df

,AEDKRW내역_종가,AEDKRW내역_오픈,AEDKRW내역_고가,AEDKRW내역_저가,AEDKRW내역_거래량,AEDKRW내역_변동,ATX내역_종가,ATX내역_오픈,ATX내역_고가,ATX내역_저가,...,헝가리BUX내역_고가,헝가리BUX내역_저가,헝가리BUX내역_거래량,헝가리BUX내역_변동,호주SPASX내역_종가,호주SPASX내역_오픈,호주SPASX내역_고가,호주SPASX내역_저가,호주SPASX내역_거래량,호주SPASX내역_변동
0,351.36,354.41,355.3,350.04,0.0,-0.82,3066.84,3059.12,3099.53,3057.85,...,45494.53,44800.37,0.0,0.45,7024.3,7133.9,7133.9,7019.5,678280000,-1.54


In [1]:
from final_target import get_target_list_db_rt, get_target
from final_predict import stock_predict
from final_model import LstmNetwork
import pickle
import tensorflow as tf
import os
import pandas as pd
from pandas.tseries.offsets import BDay
from datetime import date
import pandas as pd
from final_target import get_target_list_db_rt
# from final_realtime import realtime_trading, ds_account_db_update
from final_dbconnect import DBConnection_trading
from keras.models import load_model

# account_name, _ = ds_account_db_update(DBConnection_trading().get_sqlalchemy_connect_ip())
yesterday=str(date.today() - BDay(1)).replace('-','').split(' ')[0]


# tf gpu 메모리 관련 코드 
gpus = tf.config.list_physical_devices(device_type = 'GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


kospi_list, kosdaq_list = get_target_list_db_rt()
# investing_df = get_target(path=r'D:\systrader-dev\trading\download\target') # 각자 컴퓨터의 맞게 변경
investing_df = pd.read_csv(f'./download/investing_df/investing_{yesterday}.csv', encoding='utf-8-sig', index_col=0)

# # C:\big15\project-dev\trading\download\target
# # D:\systrader-dev\trading\download\target


# # 대형주 1년 학습
# account_name= '04BIG15'
# ls_big_year = LstmNetwork('대형주', kospi_list, '1년', 280, 0.02)
# col_list_big_year = ls_big_year.col_list
# model_big_year = ls_big_year.model_weight

# # 대형주 1년 트레이딩
# stock_predict(kospi_list, investing_df, col_list_big_year, model_big_year, account_name)



# # 대형주 6개월 학습
account_name= '03big15'
ls_big_six_month = LstmNetwork('대형주', kospi_list, '6개월', 140, 0.04 )

# with open(f'./pickle/pickle_corr_complete/대형주_20221221_6개월_10개_0.04.pkl', 'rb') as f:
#     col_list_big_six_month = pickle.load(f) # 상관 계수에 따른 컬럼 리스트
# model_big_six_month = load_model(f"./model/weight/대형주_20221221_6개월_lstm_30ep_64bs_5pa_0.04newcor.hdf5")
col_list_big_six_month = ls_big_six_month.col_list
model_big_six_month = ls_big_six_month.model_weight

# with open('./pickle/')

# 대형주 6개월 트레이딩


실시간 트레이딩 기준 날짜 : 20221221
False


100%|██████████| 10/10 [00:43<00:00,  4.39s/it]


학습 모델 저장 완료


1287it [00:13, 96.50it/s]
100%|██████████| 6/6 [00:00<?, ?it/s]


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 19, 1)]           0         
_________________________________________________________________
lstm (LSTM)                  (None, 19, 16)            1152      
_________________________________________________________________
batch_normalization (BatchNo (None, 19, 16)            64        
_________________________________________________________________
activation (Activation)      (None, 19, 16)            304       
_________________________________________________________________
lstm_1 (LSTM)                (None, 16)                2112      
_________________________________________________________________
batch_normalization_1 (Batch (None, 16)                64        
_________________________________________________________________
activation_1 (Activation)    (None, 16)                16    

KeyboardInterrupt: 

In [2]:
stock_predict(kospi_list, investing_df, col_list_big_six_month, model_big_six_month, account_name)


SQL 에러 발생


c:\Users\TJ\anaconda3\envs\tf2\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


ValueError: X has 49 features, but MaxAbsScaler is expecting 28 features as input.

In [15]:
def update(df, conn):
    args = df.values.tolist()

    sql_update = f'INSERT INTO 01big15_account_status VALUES (%s,%s,%s,%s,%s,%s,%s)'

    cursor =  conn.cursor()

    cursor.executemany(sql_update, args)
    conn.commit()


In [1]:
import pandas as pd
from final_dbconnect import DBConnection_trading, DBConnection_present



In [2]:
status_data = {'종목코드': ['dsfdsfsdfsdfsd']
                   , '종목명' : ['name_list']
                   , '보유수량' : ['amount_list']
                   , '평단가' : ['buyPrice_list']
                   , '평가금액' : ['evalValue_list']
                   , '수익율' : ['evalPerc_list']
                   , '장부금액' : ['current_value']
                   }
status_df = pd.DataFrame(status_data)

status_db_df = status_df.copy()
status_db_df.rename(columns={'종목코드': 'code', '종목명' : 'name', '보유수량' : 'amount', '평단가' : 'buyprice'
                            , '평가금액' : 'evalValue' , '수익율' : 'ratio', '장부금액' : 'currentValue'}, inplace=True)

In [16]:
update(status_db_df, DBConnection_present().get_pymysql_connection())

In [5]:
args = status_db_df.values.tolist()

args

[['dsfdsfsdfsdfsd',
  'name_list',
  'amount_list',
  'buyPrice_list',
  'evalValue_list',
  'evalPerc_list',
  'current_value']]

In [11]:

sql_update = 'INSERT INTO web_data.01big15_account_status VALUES (%s,%s,%s,%s,%s,%s,%s)'


In [14]:

conn = DBConnection_present().get_pymysql_connection()
cursor =  conn.cursor()

print(cursor.executemany(sql_update, args))
conn.commit()


1


In [15]:
from final_dbconnect import DBConnection_present
from datetime import datetime

def trading_history(conn, account_name, code, num, amount):
    now = datetime.now()
    time_ = str(now.year) + '-' +  str(now.month) + '-' + str(now.day) + ' ' + str(now.hour) + ':' + str(now.minute)
    print(time_)

    account_name = account_name.lower()
    args = [(str(account_name), str(code), str(num), str(amount), str(time_))]  # 계좌이름, 종목코드, 매수, 매도

    try: 
        sql_create = f'create table web_data.{account_name}_history(account_name varchar(20), stock_code varchar(20), buy_num varchar(20), sell_num varchar(20), his_time varchar(50))'
        cur = conn.cursor()
        cur.execute(sql_create)
        conn.commit()
        
    except:
        print('에러')

    sql_update = f"INSERT INTO web_data.{account_name}_history VALUES (%s,%s,%s,%s,%s)"
    print(sql_update)
    cursor = conn.cursor()
    cursor.executemany(sql_update, args)

    conn.commit()
    conn.close()


In [16]:
trading_history(DBConnection_present().get_pymysql_connection(), 'big15_history', 'aaaaa', 12, 13)

2022-12-26 11:15
에러
INSERT INTO web_data.big15_history_history VALUES (%s,%s,%s,%s,%s)
